In [1]:
import pandas as pd
from datetime import date, timedelta

In [2]:
from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [3]:
day = date(2024, 3, 1)
day_dateoffset = day - pd.DateOffset(months=1)
print(day_dateoffset)

2024-02-01 00:00:00


In [4]:
day = date(2024, 3, 1)
day_timedelta = day - timedelta(days=31)
print(day_timedelta)

2024-01-30


In [5]:
f = open('call_rate.csv', 'r', encoding='utf-8')

# 데이터를 완전히 동일하게 가져올 수 있는가
headers = f.readline().rstrip() # 헤더 읽기, 맨 끝 줄바꿈기호 삭제
print(headers.split('\t'))
headers = headers.split('\t') # 같은 이름의 변수에 리스트로 바꾼 것 넣어주기

# 데이터 읽기
lines = f.readlines()
print(lines[:5])

new_datas = []
for line in lines: # 한 줄씩 
    data = line.rstrip().split('\t') # 줄바꿈 기호 지우고, ,로 잘라준다.
    dict_data = { header: data[i] for i, header in enumerate(headers) }
    new_datas.append(dict_data) # 데이터를 딕셔너리로 만들어서 넣어야된다.

f.close()

['date', 'price', 'label']
['2024-03-21\t3.5290\t-0.11\n', '2024-03-20\t3.5330\t0.56\n', '2024-03-19\t3.5130\t2.09\n', '2024-03-18\t3.4410\t-0.66\n', '2024-03-15\t3.4640\t-0.63\n']


In [6]:
def find_nearest_date(df, target_date):
    delta = pd.Timedelta(days=1)
    nearest_date = None,
    min_difference = pd.Timedelta.max

    for date in df['date']:
        difference = abs(target_date - date)
        if difference < min_difference:
            min_difference = difference
            nearest_date = date

    return nearest_date

def label_call(df):
    df_ud = pd.DataFrame(index=df.index, columns=['date', 'price', 'label'])

    # 'date' 열을 datetime 형식으로 변환
    df['date'] = pd.to_datetime(df['date'])

    for i, row in df.iterrows():
        time = row['date']
        time2 = time - pd.DateOffset(months=1)

        # 이전 달의 날짜가 인덱스에 있는지 확인
        if time2 in df['date'].values:
            prev_row = df[df['date'] == time2].iloc[0]
        else:
            # 이전 날짜가 없는 경우 가장 가까운 날짜로 설정
            nearest_date = find_nearest_date(df, time2)
            prev_row = df[df['date'] == nearest_date].iloc[0]

        if row['price'] > prev_row['price']:
            label = '1'
        elif row['price'] == prev_row['price']:
            label = 0
        else:
            label = '-1'

        df_ud.loc[i] = [time.date(), row['price'], label]

    return df_ud 

In [7]:
df = pd.DataFrame(new_datas)

call = label_call(df)
call

,date,price,label
0,2024-03-21,3.5290,-1
1,2024-03-20,3.5330,-1
2,2024-03-19,3.5130,-1
3,2024-03-18,3.4410,-1
4,2024-03-15,3.4640,-1
...,...,...,...
3253,2011-01-07,2.4900,0
3254,2011-01-06,2.4900,0
3255,2011-01-05,2.4900,0
3256,2011-01-04,2.5000,1


In [ ]:
pip install ekonlpy

In [9]:
import konlpy as kn
import MeCab as mc
import ekonlpy as ek
import pandas as pd
import numpy as np
import re
from pandas import Series, DataFrame 

In [10]:
# 크롤링 결과물 불러오기 - 이데일리 뉴스 
from ekonlpy.tag import Mecab
from ekonlpy.sentiment import MPCK
mpck = MPCK()

# 패턴 정의
news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/edaily_news_test.json'
df = pd.read_json(result_path)
df = df.sort_values(by=["date"], ascending=False) # 날짜별 내림차순

# 특정패턴 제거
for pattern in news_pattern:
    df['content'] = df['content'].apply(lambda x: re.sub(pattern, '', x))
news1_df = df

news1_df['words']=" "  # 빈 값을 가진 words column 추가
for i, row in df.iterrows() :
    try :
        tmp_df = news1_df['content']
        #임시 데이터프레임에 불러온 기사의 본문 내용 리스트로 저장
        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']
        #eKoNLPy로 tokenizing 한 것을 데이터프레임에 column 추가

    except Exception as ex :
        print("fail")
        
news1_df = news1_df.drop(['content', 'title'], axis=1)  # axis=1은 열을 의미함
news1_df.columns

Index(['date', 'words'], dtype='object')

In [12]:
# 크롤링 결과물 불러오기 - 중앙일보 뉴스 
mpck = MPCK()

news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/joongamg_news_test.json'
df = pd.read_json(result_path)
df = df.sort_values(by=["date"], ascending=False)

for pattern in news_pattern:
    df['content'] = df['content'].apply(lambda x: re.sub(pattern, '', x))
news2_df = df

news2_df['words']=" "
for i, row in df.iterrows() :
    try :
        tmp_df = news2_df['content']

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']

    except Exception as ex :
        print("fail")

news2_df = news2_df.drop(['content', 'title'], axis=1)

Index(['date', 'words'], dtype='object')

In [14]:
# 크롤링 결과물 불러오기 - 머니투데이 뉴스 
mpck = MPCK()

news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/moneytoday_news_test.json'
df = pd.read_json(result_path)
df = df.sort_values(by=["date"], ascending=False) 

for pattern in news_pattern:
    df['content'] = df['content'].apply(lambda x: re.sub(pattern, '', x))
news3_df = df

news3_df['words']=" " 
for i, row in df.iterrows() :

    try :
        tmp_df = news3_df['content']

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']

    except Exception as ex :
        print("fail")

news3_df = news3_df.drop(['content', 'title'], axis=1)

Index(['date', 'words'], dtype='object')

In [16]:
# 크롤링 결과물 불러오기 - 의사록 
mpck = MPCK()

news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/minutes_re.tsv'
df = pd.read_csv(result_path, sep='\t')

for pattern in news_pattern:
    df['내용'] = df['내용'].apply(lambda x: re.sub(pattern, '', x))
min_df = df

min_df['words']=" "
for i, row in df.iterrows() :
    try :
        tmp_df = min_df['내용']

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']

    except Exception as ex :
        print("fail")

min_df = min_df.drop('내용', axis=1) 
min_df = min_df.rename(columns={'날짜': 'date'})

Index(['날짜', 'words'], dtype='object')

In [18]:
# 크롤링 결과물 불러오기 - 채권 보고서 
mpck = MPCK()

news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/text_by_date.json1_test.json'
df = pd.read_json(result_path)
df = df.sort_values(by=["date"], ascending=False)

for pattern in news_pattern:
    df['text'] = df['text'].apply(lambda x: re.sub(pattern, '', x))
bond_df = df

bond_df['words']=" "
for i, row in df.iterrows() :
    try :
        tmp_df = bond_df['text']

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']

    except Exception as ex :
        print("fail")

bond_df = bond_df.drop('text', axis=1)


Index(['date', 'words'], dtype='object')

In [20]:
# 전체 크롤링 데이터들 -> 데이터프레임으로 합치기
crawl = pd.concat([news1_df, news2_df, news3_df, min_df, bond_df])
crawl['date'] = pd.to_datetime(crawl['date']).dt.date
crawl

,date,words
298,2023-05-04,fed/NNG.fed/NNG.금리/NNG.포인트/NNG.인상/NNG.하/XSV.가운...
288,2023-05-04,fed/NNG.fed/NNG.금리/NNG.인상/NNG.종료/NNG.시사/NNG.하/...
297,2023-05-04,fed/NNG.fed/NNG.매파/NNG.색채/NNG.드러나/VV.fed/NNG.예...
296,2023-05-04,fed/NNG.fed/NNG.fed/NNG.금리/NNG.수준/NNG.인상/NNG.하...
287,2023-05-04,fed/NNG.fed/NNG.현지/NNG.개최/NNG.달/NNG.fed/NNG.fe...
...,...,...
4,2011-01-07,금리/NNG.국고채/NNG.금리/NNG.추이/NNG.일드커브/NNG.형태/NNG.변...
3,2011-01-04,거래주체/NNG.매매동향/NNG.은행/NNG.순매수/NNG.상위/NNG.은행/NNG...
2,2011-01-02,거래주체/NNG.매매동향/NNG.은행/NNG.순매수/NNG.상위/NNG.은행/NNG...
1,2011-01-01,에너지기업/NNG.유가/NNG.급락/NNG.트레이드/NNG.증권/NNG.본부/NNG...


In [22]:
# # korean stop words 불러오기
# stopwords_path = 'korean_sotp_words.txt'
# stopwords = []
# with open(stopwords_path, 'r', encoding='utf-8') as file:
#     for line in file:
#         stopwords.append(line.strip())
# print(stopwords)  # 불용어 모음

In [23]:
# word = []
# for tag in text: #토큰화된 text
#     if tag[1] not in stopwords:
#         if tag[0] not in stopwords:
#             word.append(tag[0])

In [24]:
# 품사 태깅 제거
import re

def extract_hangul_words(text):
    words = ' '.join(re.findall('[가-힣]+', text))
    return words

crawl['words'] = crawl['words'].apply(extract_hangul_words)

In [26]:
# 콜금리에 date기준으로 크롤링 칼럼 추가하기
import pandas as pd

# Date 열을 datetime으로 변환하고 인덱스로 설정
call['date'] = pd.to_datetime(call['date'])
call.set_index('date', inplace=True)

crawl['date'] = pd.to_datetime(crawl['date'])
crawl.set_index('date', inplace=True)

# 중복된 인덱스 제거
call = call[~call.index.duplicated()]
crawl = crawl[~crawl.index.duplicated()]

# 동일한 날짜에 대해 열을 합치기
merged_df = pd.concat([call, crawl], axis=1)

# 'date' 열을 다시 열로 변환
merged_df.reset_index(inplace=True)

# 결과 출력
print(merged_df.columns)
print(merged_df)

Index(['date', 'price', 'label', 'words'], dtype='object')
           date   price label  \
0    2006-05-11     NaN   NaN   
1    2006-05-25     NaN   NaN   
2    2006-06-08     NaN   NaN   
3    2006-06-22     NaN   NaN   
4    2006-07-07     NaN   NaN   
...         ...     ...   ...   
3401 2024-03-15  3.4640    -1   
3402 2024-03-18  3.4410    -1   
3403 2024-03-19  3.5130    -1   
3404 2024-03-20  3.5330    -1   
3405 2024-03-21  3.5290    -1   

                                                  words  
0     조 의거 하 의안 통화정책 방향 상정 하 토의 내용 국내외 경제동향 하 국제 유가 ...  
1     담당 보가 지급 결제 원활 위하 금융기관 일중 당좌대출 한도규제 폐지 하 일중 당좌...  
2     조 의거 하 의안 통화정책 방향 상정 하 토의 내용 국내외 경제동향 하 경기 선행 ...  
3     조건 구비 하 사항 명시 하 차액 결제 참가 기관 자금 조정 적수 보전 이자 정산 ...  
4     보 금융시장 동향 대하 보고 내용 별첨 조 의거 하 의안 통화정책 방향 상정 하 토...  
...                                                 ...  
3401  가명 아이 낳 신생아 특례 대출 받 대기업 근무 하 같 남편 소득 합하 달하 신생아...  
3402                                                NaN  
3403                  

In [27]:
# 'label' 칼럼의 값이 '1(매파)'인 행과 '-1(비둘기파)'인 행으로 데이터프레임 분리
df_P = merged_df[merged_df['label'] == '1']
df_N = merged_df[merged_df['label'] == '-1']

# Nan인 행 제거
df_P = df_P.dropna()
df_N = df_N.dropna()

# 필요없는 열 제거 
df_P = df_P.drop(['date', 'price'], axis=1)
df_N = df_N.drop(['date', 'price'], axis=1)

word_df = pd.concat([df_P,df_N],axis=0)
x_tain = word_df["words"]
y_tain = word_df["label"]

In [30]:
# 매파 words 빈도수 체크 
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import Counter

# 텍스트 칼럼에서 단어 횟수 세기
text = ' '.join(df_P['words'])  # 텍스트 칼럼의 모든 텍스트를 하나의 문자열로 결합
tokens = word_tokenize(text)  # 텍스트를 단어로 토큰화
word_counts = Counter(tokens)  # 단어 등장 횟수 카운트

# 빈도수가 15 이상인 단어 필터링
filtered_word_counts = {word: count for word, count in word_counts.items() if count >= 15}

df_P_counts = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
df_P_counts['P'] = 1 # 라벨링 다시 추가

,Word,Frequency
0,은행,843
1,순매수,153
2,상위,93
3,순매도,132
4,보험기금,22
...,...,...
1698,병목,30
1699,오미크론,27
1700,우크라이나사태,77
1701,빈일자리,26


In [33]:
# df_P_counts.to_csv("hawkish_count_1.csv", mode='w')

In [34]:
# 비둘기파 words 빈도수 체크
text = ' '.join(df_N['words'])  # 텍스트 칼럼의 모든 텍스트를 하나의 문자열로 결합
tokens = word_tokenize(text)  # 텍스트를 단어로 토큰화
word_counts = Counter(tokens)  # 단어 등장 횟수 카운트

# 빈도수가 15 이상인 단어 필터링
filtered_word_counts = {word: count for word, count in word_counts.items() if count >= 15}

# 데이터프레임 생성
df_N_counts = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
df_N_counts['N'] = 1

,Word,Frequency
0,표,37
1,만기,157
2,회사채,440
3,종목,24
4,신용등급,94
...,...,...
1939,대면,21
1940,변이,17
1941,자녀,24
1942,전기차,37


In [37]:
# df_B_counts.to_csv("dovish_count_1.csv", mode='w')

In [42]:
# 2-그램
from collections import defaultdict
import pandas as pd

def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        words = row.split()
        ngrams.extend([' '.join(words[i:i+n]) for i in range(len(words)-n+1)])
    return ngrams

def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

n = 2
ngrams = generate_ngrams(df_A['words'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 15 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 15}

# 데이터프레임으로 변환
selected_ngrams_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
selected_ngrams_df['N'] = 1
selected_ngrams_df 

,ngram,frequency,N
0,순매수 상위,44,1
1,순매도 상위,44,1
2,상위 보험기금,22,1
3,상위 투신,22,1
4,상위 외국인,22,1
...,...,...,...
3083,물가 둔화,17,1
3084,목 목,18,1
3085,리 오프닝,33,1
3086,인플레이션 둔화,31,1
